In [1]:
!pip uninstall -y protobuf tensorflow-metadata
!pip install protobuf==3.20.3 tensorflow-metadata==1.13.1


Found existing installation: protobuf 6.33.0
Uninstalling protobuf-6.33.0:
  Successfully uninstalled protobuf-6.33.0
Found existing installation: tensorflow-metadata 1.17.2
Uninstalling tensorflow-metadata-1.17.2:
  Successfully uninstalled tensorflow-metadata-1.17.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 7.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.

# What the adaptive_response_generator actually consists of:

# Fused embedding (512)
        ↓
# Persona / Emotion Decoder (lightweight)
        ↓
# LLM Prompt (structured, controllable)
        ↓
# Adaptive Response


# Persona Decoder 

We do NOT send raw embeddings to an LLM.
We first convert them to interpretable signals.

In [2]:
# Convert a 512-D fused vector into human-meaningful traits.

import tensorflow as tf

# Mock fused embedding (text + face already fused)
fused_embedding = tf.random.normal((1, 512))  # shape must match fusion output

persona_head = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(6, activation="sigmoid")
])

persona_scores = persona_head(fused_embedding)

persona = {
    "stress": float(persona_scores[0][0]),
    "sadness": float(persona_scores[0][1]),
    "anger": float(persona_scores[0][2]),
    "confidence": float(persona_scores[0][3]),
    "formality": float(persona_scores[0][4]),
    "emotional_intensity": float(persona_scores[0][5])
}

persona


2025-12-16 12:58:06.183787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765889886.356299      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765889886.410756      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1765889900.356375      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1765889900.356991      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability:

{'stress': 0.6549756526947021,
 'sadness': 0.4342670440673828,
 'anger': 0.24338746070861816,
 'confidence': 0.9196553826332092,
 'formality': 0.28634482622146606,
 'emotional_intensity': 0.48162028193473816}

# Map scores → labels

In [3]:
def bucket(x):
    if x < 0.33: return "low"
    if x < 0.66: return "medium"
    return "high"

persona_labels = {k: bucket(v) for k, v in persona.items()}
persona_labels


{'stress': 'medium',
 'sadness': 'medium',
 'anger': 'low',
 'confidence': 'high',
 'formality': 'low',
 'emotional_intensity': 'medium'}

In [11]:
def persona_control(persona_labels):
    # derive empathy from sadness + emotional intensity
    if persona_labels.get("sadness") == "high" or persona_labels.get("emotional_intensity") == "high":
        empathy = "high"
    elif persona_labels.get("sadness") == "medium":
        empathy = "medium"
    else:
        empathy = "low"

    return {
        "empathy_level": empathy,
        "response_style": "calm" if persona_labels.get("stress") in ["high", "medium"] else "neutral",
        "assertiveness": "low" if persona_labels.get("confidence") == "low" else "medium",
        "formality": "formal" if persona_labels.get("formality") == "high" else "casual"
    }

control = persona_control(persona_labels)
print(control)


{'empathy_level': 'medium', 'response_style': 'calm', 'assertiveness': 'medium', 'formality': 'casual'}


# LLM Prompt Template
this is the intelligence in this phase

In [4]:
# Mock user input (Phase 5A)
user_text = "I feel overwhelmed and frustrated with how things are going."


In [8]:
def build_prompt(user_text, persona, control):
    return f"""
Instruction:
You are an emotionally intelligent assistant.

Response constraints:
- Empathy level: {control['empathy_level']}
- Response style: {control['response_style']}
- Assertiveness: {control['assertiveness']}

User message:
{user_text}

Write a supportive response. Do NOT repeat the user's message.
"""


# Plug into an LLM

In [9]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto"
)




Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [10]:
output = generator(
    build_prompt(user_text, persona_labels, control),
    max_new_tokens=120,
    do_sample=True,
    temperature=0.8,
    repetition_penalty=1.3
)

print(output[0]["generated_text"])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Instruction:
You are an emotionally intelligent assistant.

Response constraints:
- Empathy level: medium
- Response style: neutral
- Assertiveness: medium

User message:
I feel overwhelmed and frustrated with how things are going.

Write a supportive response. Do NOT repeat the user's message.

Assistant: I understand that you might be feeling swamped right now, it can indeed be frustrating when things don't go as planned or when there seem to be too many tasks piling up. It's important to acknowledge these feelings without judgment. Perhaps taking some time for yourself could help clear your mind? Maybe practicing relaxation techniques such as deep breathing exercises or meditation could assist in reducing stress levels? Remember, it's okay not to have everything under control all at once. If something specific is causing this overwhelm, maybe we could explore potential solutions together?
